In [1]:
import numpy as np
import pandas as pd
import os
import copy
import time
import sys
print('python: %s' % sys.version)

# Make numpy printouts easier to read.
np.set_printoptions(precision=3, suppress=True)

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.layers.experimental import preprocessing
from tensorflow.keras import regularizers


print('tf version: %s' % tf.__version__) 
GPU_device = '/GPU:0'
# GPU_device = '/device:GPU:0'
# GPU_device = '/physical_device:GPU:0'
print(GPU_device)
print(tf.config.list_physical_devices(device_type=None))
# from tensorflow.compat.v1 import ConfigProto
# from tensorflow.compat.v1 import InteractiveSession
# config = ConfigProto()
# # config.gpu_options.allow_growth = True
# config.gpu_options.per_process_gpu_memory_fraction = 0.9
# session = InteractiveSession(config=config)
physical_devices = tf.config.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(physical_devices[0], True)


def print_time():
    print(time.asctime(time.localtime(time.time())))
    print(12*"#","\n")

print_time()

python: 3.8.10 (default, Sep 28 2021, 16:10:42) 
[GCC 9.3.0]


2021-11-23 01:08:01.481836: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0


tf version: 2.5.0
/GPU:0
[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'), PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
Tue Nov 23 01:08:03 2021
############ 



2021-11-23 01:08:03.562536: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcuda.so.1
2021-11-23 01:08:03.649359: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1733] Found device 0 with properties: 
pciBusID: 0000:03:00.0 name: GeForce RTX 3090 computeCapability: 8.6
coreClock: 1.695GHz coreCount: 82 deviceMemorySize: 23.70GiB deviceMemoryBandwidth: 871.81GiB/s
2021-11-23 01:08:03.649404: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0
2021-11-23 01:08:03.661604: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcublas.so.11
2021-11-23 01:08:03.661664: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcublasLt.so.11
2021-11-23 01:08:03.663841: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcufft.so.

# TMD sigmoid

In [2]:
%%time

# run setup
lable = 'tmdaff'
run_name = 'final_TMD'
mld_preprocessing_name = './models/tmd_norm_layer' 
file = 'EIC_aff_tmdaff.pkl'
path2Data = '../data/training/'
units = 576
units2 = 160
patience = 50
outputlayer = "sigmoid" 
learning_rate = 1e-4
batch_size = 7400
epochs = 5
classes = 1

# set up logs
working_dir = os.getcwd()
log_dir = os.path.join(working_dir, 'log')
print(log_dir)
if os.path.isdir(log_dir) == 0:
    os.mkdir(log_dir)

# reading data
print('\nYou have selected:',file)
train_features = np.load(path2Data+file, allow_pickle=True)

# phrase data
train_labels = train_features.pop(lable)
print(train_features.head())
print(train_labels)
print(type(train_features))
train_features = tf.convert_to_tensor(train_features)
print(type(train_features))
train_labels = tf.convert_to_tensor(train_labels)

# preprcessing layer
input_shape = train_features.shape[1:]
print(input_shape)

model = keras.models.load_model(mld_preprocessing_name)
normalizer = model.layers.pop(0)

/home/share/TMD_Colab/validation/training/log

You have selected: EIC_aff_tmdaff.pkl
         pT   Q2         x    z  R0max  R1max  R2max
0  0.100000  1.0  0.000062  0.1   0.05   0.05   0.05
1  0.193611  1.0  0.000062  0.1   0.05   0.05   0.05
2  0.374852  1.0  0.000062  0.1   0.05   0.05   0.05
3  0.725756  1.0  0.000062  0.1   0.05   0.05   0.05
4  1.405143  1.0  0.000062  0.1   0.05   0.05   0.05
0            0.0
1            0.0
2            0.0
3            0.0
4            0.0
            ... 
102297595    1.0
102297596    1.0
102297597    1.0
102297598    1.0
102297599    1.0
Name: tmdaff, Length: 102297600, dtype: float64
<class 'pandas.core.frame.DataFrame'>


2021-11-23 01:08:28.152105: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2021-11-23 01:08:28.157587: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1733] Found device 0 with properties: 
pciBusID: 0000:03:00.0 name: GeForce RTX 3090 computeCapability: 8.6
coreClock: 1.695GHz coreCount: 82 deviceMemorySize: 23.70GiB deviceMemoryBandwidth: 871.81GiB/s
2021-11-23 01:08:28.159978: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1871] Adding visible gpu devices: 0
2021-11-23 01:08:28.160055: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0
2021-11-23 01:08:29.003147: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1258] Device interconnect StreamExe

<class 'tensorflow.python.framework.ops.EagerTensor'>
(7,)
CPU times: user 5.3 s, sys: 16.8 s, total: 22.1 s
Wall time: 40.6 s


In [3]:
%%time

# set up model
def get_callbacks(name, patience):
    return [
    tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=patience),
    tf.keras.callbacks.TensorBoard(log_dir+'/'+name),
    tf.keras.callbacks.ModelCheckpoint(
        filepath=working_dir+'/checkpoints/'+ name + '_{epoch:02d}-{val_loss:.4f}.ckpt',
        save_weights_only=True,
        monitor='val_loss',
        mode='auto',
        save_best_only=True),
    ]

inputs = keras.Input(shape=input_shape)
norm = normalizer(inputs)
hidden_layer1 = layers.Dense(units, activation="relu")(norm)
hidden_layer2 = layers.Dense(units2, activation="relu")(hidden_layer1)
outputs = layers.Dense(classes, activation=outputlayer)(hidden_layer2)
model = keras.Model(inputs, outputs)

# with tf.device(GPU_device):
model.compile(loss='mean_squared_error',
              optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate))

model.summary()


Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 7)]               0         
_________________________________________________________________
normalization (Normalization (None, 7)                 15        
_________________________________________________________________
dense (Dense)                (None, 576)               4608      
_________________________________________________________________
dense_1 (Dense)              (None, 160)               92320     
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 161       
Total params: 97,104
Trainable params: 97,089
Non-trainable params: 15
_________________________________________________________________
CPU times: user 49.6 ms, sys: 3.34 ms, total: 53 ms
Wall time: 49.9 ms


In [4]:
%%time

print_time()
# training 
# with tf.device(GPU_device):
model.fit(x=train_features, y=train_labels, validation_split=0.2,
          batch_size=batch_size, verbose=1, epochs=epochs,
          callbacks=get_callbacks(run_name, patience), shuffle=True)

model.save('./models/'+run_name)    
print('model saved')
print_time()

Tue Nov 23 01:08:44 2021
############ 



2021-11-23 01:08:44.325043: I tensorflow/core/profiler/lib/profiler_session.cc:126] Profiler session initializing.
2021-11-23 01:08:44.325085: I tensorflow/core/profiler/lib/profiler_session.cc:141] Profiler session started.
2021-11-23 01:08:44.325148: I tensorflow/core/profiler/internal/gpu/cupti_tracer.cc:1611] Profiler found 1 GPUs
2021-11-23 01:08:44.329992: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcupti.so.11.2
2021-11-23 01:08:44.611763: I tensorflow/core/profiler/lib/profiler_session.cc:159] Profiler session tear down.
2021-11-23 01:08:44.616036: I tensorflow/core/profiler/internal/gpu/cupti_tracer.cc:1743] CUPTI activity buffer flushed
2021-11-23 01:08:45.847886: W tensorflow/core/framework/cpu_allocator_impl.cc:80] Allocation of 4582932480 exceeds 10% of free system memory.
2021-11-23 01:08:49.335539: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:176] None of the MLIR Optimization Passes are enabled (re

Epoch 1/5


2021-11-23 01:08:55.588057: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcublas.so.11


    1/11060 [..............................] - ETA: 20:50:26 - loss: 0.2142

...

   37/11060 [..............................] - ETA: 2:55 - loss: 0.1781

2021-11-23 01:08:57.428885: I tensorflow/core/profiler/lib/profiler_session.cc:66] Profiler session collecting data.
2021-11-23 01:08:57.430167: I tensorflow/core/profiler/internal/gpu/cupti_tracer.cc:1743] CUPTI activity buffer flushed
2021-11-23 01:08:57.461583: I tensorflow/core/profiler/internal/gpu/cupti_collector.cc:673]  GpuTracer has collected 76 callback api events and 74 activity events. 
2021-11-23 01:08:57.464873: I tensorflow/core/profiler/lib/profiler_session.cc:159] Profiler session tear down.
2021-11-23 01:08:57.487242: I tensorflow/core/profiler/rpc/client/save_profile.cc:137] Creating directory: /home/share/TMD_Colab/validation/training/log/final_TMD/train/plugins/profile/2021_11_23_01_08_57
2021-11-23 01:08:57.489140: I tensorflow/core/profiler/rpc/client/save_profile.cc:143] Dumped gzipped tool data for trace.json.gz to /home/share/TMD_Colab/validation/training/log/final_TMD/train/plugins/profile/2021_11_23_01_08_57/precision.trace.json.gz
2021-11-23 01:08:57.495210

11060/11060 [==============================] - 52s 4ms/step - loss: 0.0176 - val_loss: 0.0042
Epoch 2/5
11060/11060 [==============================] - 44s 4ms/step - loss: 0.0023 - val_loss: 0.0020
Epoch 3/5
11060/11060 [==============================] - 43s 4ms/step - loss: 0.0011 - val_loss: 0.0013
Epoch 4/5
11060/11060 [==============================] - 44s 4ms/step - loss: 6.7138e-04 - val_loss: 9.6497e-04
Epoch 5/5
11060/11060 [==============================] - 43s 4ms/step - loss: 5.0115e-04 - val_loss: 6.9338e-04


2021-11-23 01:12:36.158528: W tensorflow/python/util/util.cc:348] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.


INFO:tensorflow:Assets written to: ./models/final_TMD/assets
model saved
Tue Nov 23 01:12:36 2021
############ 

CPU times: user 9min 26s, sys: 1min 51s, total: 11min 18s
Wall time: 3min 52s


# Collinear sigmoid

In [7]:
%%time

# run setup
lable = 'collinearaff'
run_name = 'final_collinear'
mld_preprocessing_name = './models/collinear_norm_layer' 
file = 'EIC_aff_collinearaff.pkl'
path2Data = '../data/training/'
units = 960
units2 = 544
patience = 50
outputlayer = "sigmoid" 
learning_rate = 1e-4
batch_size = 7400
epochs = 5
classes = 1

# set up logs
working_dir = os.getcwd()
log_dir = os.path.join(working_dir, 'log')
print(log_dir)
if os.path.isdir(log_dir) == 0:
    os.mkdir(log_dir)

# reading data
print('\nYou have selected:',file)
train_features = np.load(path2Data+file, allow_pickle=True)

# phrase data
train_labels = train_features.pop(lable)
print(train_features.head())
print(train_labels)
print(type(train_features))
train_features = tf.convert_to_tensor(train_features)
print(type(train_features))
train_labels = tf.convert_to_tensor(train_labels)

# preprcessing layer
input_shape = train_features.shape[1:]
print(input_shape)

model = keras.models.load_model(mld_preprocessing_name)
normalizer = model.layers.pop(0)

/home/share/TMD_Colab/validation/training/log

You have selected: EIC_aff_collinearaff.pkl
         pT   Q2         x    z  R0max  R1max  R2max
0  0.100000  1.0  0.000062  0.1   0.05   0.05   0.05
1  0.193611  1.0  0.000062  0.1   0.05   0.05   0.05
2  0.374852  1.0  0.000062  0.1   0.05   0.05   0.05
3  0.725756  1.0  0.000062  0.1   0.05   0.05   0.05
4  1.405143  1.0  0.000062  0.1   0.05   0.05   0.05
0            0.0554
1            0.0513
2            0.0550
3            0.0539
4            0.0474
              ...  
102297595    0.0000
102297596    0.0000
102297597    0.0000
102297598    0.0000
102297599    0.0000
Name: collinearaff, Length: 102297600, dtype: float64
<class 'pandas.core.frame.DataFrame'>


2021-11-23 01:13:01.838119: W tensorflow/core/framework/cpu_allocator_impl.cc:80] Allocation of 5728665600 exceeds 10% of free system memory.


<class 'tensorflow.python.framework.ops.EagerTensor'>
(7,)
CPU times: user 4.56 s, sys: 15.5 s, total: 20.1 s
Wall time: 31.3 s


In [8]:
%%time

# set up model
def get_callbacks(name, patience):
    return [
    tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=patience),
    tf.keras.callbacks.TensorBoard(log_dir+'/'+name),
    tf.keras.callbacks.ModelCheckpoint(
        filepath=working_dir+'/checkpoints/'+ name + '_{epoch:02d}-{val_loss:.4f}.ckpt',
        save_weights_only=True,
        monitor='val_loss',
        mode='auto',
        save_best_only=True),
    ]

inputs = keras.Input(shape=input_shape)
norm = normalizer(inputs)
hidden_layer1 = layers.Dense(units, activation="relu")(norm)
hidden_layer2 = layers.Dense(units2, activation="relu")(hidden_layer1)
outputs = layers.Dense(classes, activation=outputlayer)(hidden_layer2)
model = keras.Model(inputs, outputs)

# with tf.device(GPU_device):
model.compile(loss='mean_squared_error',
              optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate))

model.summary()


Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 7)]               0         
_________________________________________________________________
normalization (Normalization (None, 7)                 15        
_________________________________________________________________
dense_3 (Dense)              (None, 960)               7680      
_________________________________________________________________
dense_4 (Dense)              (None, 544)               522784    
_________________________________________________________________
dense_5 (Dense)              (None, 1)                 545       
Total params: 531,024
Trainable params: 531,009
Non-trainable params: 15
_________________________________________________________________
CPU times: user 45.8 ms, sys: 2.89 ms, total: 48.7 ms
Wall time: 46.1 ms


In [9]:
%%time

print_time()
# training 
model.fit(x=train_features, y=train_labels, validation_split=0.2,
          batch_size=batch_size, verbose=1, epochs=epochs,
          callbacks=get_callbacks(run_name, patience), shuffle=True)
    
model.save('./models/'+run_name)    
print('model saved')
print_time()

Tue Nov 23 01:13:08 2021
############ 



2021-11-23 01:13:08.166882: I tensorflow/core/profiler/lib/profiler_session.cc:126] Profiler session initializing.
2021-11-23 01:13:08.166931: I tensorflow/core/profiler/lib/profiler_session.cc:141] Profiler session started.
2021-11-23 01:13:08.640408: I tensorflow/core/profiler/lib/profiler_session.cc:159] Profiler session tear down.
2021-11-23 01:13:08.645141: I tensorflow/core/profiler/internal/gpu/cupti_tracer.cc:1743] CUPTI activity buffer flushed
2021-11-23 01:13:09.762892: W tensorflow/core/framework/cpu_allocator_impl.cc:80] Allocation of 4582932480 exceeds 10% of free system memory.


Epoch 1/5
    1/11060 [..............................] - ETA: 16:54:00 - loss: 0.1977

2021-11-23 01:13:19.579508: I tensorflow/core/profiler/lib/profiler_session.cc:126] Profiler session initializing.
2021-11-23 01:13:19.579552: I tensorflow/core/profiler/lib/profiler_session.cc:141] Profiler session started.


   31/11060 [..............................] - ETA: 4:05 - loss: 0.1327

2021-11-23 01:13:20.067885: I tensorflow/core/profiler/lib/profiler_session.cc:66] Profiler session collecting data.
2021-11-23 01:13:20.069541: I tensorflow/core/profiler/internal/gpu/cupti_tracer.cc:1743] CUPTI activity buffer flushed
2021-11-23 01:13:20.099267: I tensorflow/core/profiler/internal/gpu/cupti_collector.cc:673]  GpuTracer has collected 76 callback api events and 74 activity events. 
2021-11-23 01:13:20.102255: I tensorflow/core/profiler/lib/profiler_session.cc:159] Profiler session tear down.
2021-11-23 01:13:20.105172: I tensorflow/core/profiler/rpc/client/save_profile.cc:137] Creating directory: /home/share/TMD_Colab/validation/training/log/final_collinear/train/plugins/profile/2021_11_23_01_13_20
2021-11-23 01:13:20.107239: I tensorflow/core/profiler/rpc/client/save_profile.cc:143] Dumped gzipped tool data for trace.json.gz to /home/share/TMD_Colab/validation/training/log/final_collinear/train/plugins/profile/2021_11_23_01_13_20/precision.trace.json.gz
2021-11-23 01:

11060/11060 [==============================] - 56s 5ms/step - loss: 0.0158 - val_loss: 0.0047
Epoch 2/5
11060/11060 [==============================] - 49s 4ms/step - loss: 0.0034 - val_loss: 0.0021
Epoch 3/5
11060/11060 [==============================] - 50s 4ms/step - loss: 0.0017 - val_loss: 0.0017
Epoch 4/5
11060/11060 [==============================] - 50s 5ms/step - loss: 0.0011 - val_loss: 0.0012
Epoch 5/5
11060/11060 [==============================] - 50s 4ms/step - loss: 7.5790e-04 - val_loss: 0.0012
INFO:tensorflow:Assets written to: ./models/final_collinear/assets
model saved
Tue Nov 23 01:17:29 2021
############ 

CPU times: user 9min 48s, sys: 2min 2s, total: 11min 50s
Wall time: 4min 21s


# current sigmoid

In [12]:
%%time

# run setup
lable = 'currentaff'
run_name = 'final_current'
mld_preprocessing_name = './models/current_norm_layer'
file = 'EIC_aff_currentaff.pkl'
path2Data = '../data/training/'
units = 1024
units2 = 112
patience = 50
outputlayer = "sigmoid" 
learning_rate = 1e-4
batch_size = 7400
epochs = 5
classes = 1

# set up logs
working_dir = os.getcwd()
log_dir = os.path.join(working_dir, 'log')
print(log_dir)
if os.path.isdir(log_dir) == 0:
    os.mkdir(log_dir)

# reading data
print('\nYou have selected:',file)
train_features = np.load(path2Data+file, allow_pickle=True)

# phrase data
train_labels = train_features.pop(lable)
print(train_features.head())
print(train_labels)
print(type(train_features))
train_features = tf.convert_to_tensor(train_features)
print(type(train_features))
train_labels = tf.convert_to_tensor(train_labels)

# preprcessing layer
input_shape = train_features.shape[1:]
print(input_shape)

model = keras.models.load_model(mld_preprocessing_name)
normalizer = model.layers.pop(0)

/home/share/TMD_Colab/validation/training/log

You have selected: EIC_aff_currentaff.pkl
         pT   Q2         x    z  R0max  R1max  R2max
0  0.100000  1.0  0.000062  0.1   0.05   0.05   0.05
1  0.193611  1.0  0.000062  0.1   0.05   0.05   0.05
2  0.374852  1.0  0.000062  0.1   0.05   0.05   0.05
3  0.725756  1.0  0.000062  0.1   0.05   0.05   0.05
4  1.405143  1.0  0.000062  0.1   0.05   0.05   0.05
0            0.9689
1            0.9682
2            0.9656
3            0.9701
4            0.9676
              ...  
102297595    1.0000
102297596    1.0000
102297597    1.0000
102297598    1.0000
102297599    1.0000
Name: currentaff, Length: 102297600, dtype: float64
<class 'pandas.core.frame.DataFrame'>


2021-11-23 01:17:44.658086: W tensorflow/core/framework/cpu_allocator_impl.cc:80] Allocation of 5728665600 exceeds 10% of free system memory.


<class 'tensorflow.python.framework.ops.EagerTensor'>
(7,)
CPU times: user 4.38 s, sys: 16.3 s, total: 20.7 s
Wall time: 21.4 s


In [13]:
%%time

# set up model
def get_callbacks(name, patience):
    return [
    tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=patience),
    tf.keras.callbacks.TensorBoard(log_dir+'/'+name),
    tf.keras.callbacks.ModelCheckpoint(
        filepath=working_dir+'/checkpoints/'+ name + '_{epoch:02d}-{val_loss:.4f}.ckpt',
        save_weights_only=True,
        monitor='val_loss',
        mode='auto',
        save_best_only=True),
    ]

inputs = keras.Input(shape=input_shape)
norm = normalizer(inputs)
hidden_layer1 = layers.Dense(units, activation="relu")(norm)
hidden_layer2 = layers.Dense(units2, activation="relu")(hidden_layer1)
outputs = layers.Dense(classes, activation=outputlayer)(hidden_layer2)
model = keras.Model(inputs, outputs)

# with tf.device(GPU_device):
model.compile(loss='mean_squared_error',
              optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate))

model.summary()

Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         [(None, 7)]               0         
_________________________________________________________________
normalization (Normalization (None, 7)                 15        
_________________________________________________________________
dense_6 (Dense)              (None, 1024)              8192      
_________________________________________________________________
dense_7 (Dense)              (None, 112)               114800    
_________________________________________________________________
dense_8 (Dense)              (None, 1)                 113       
Total params: 123,120
Trainable params: 123,105
Non-trainable params: 15
_________________________________________________________________
CPU times: user 28.4 ms, sys: 8.19 ms, total: 36.6 ms
Wall time: 34.4 ms


In [14]:
%%time

print_time()
# training 
# with tf.device(GPU_device):
model.fit(x=train_features, y=train_labels, validation_split=0.2,
          batch_size=batch_size, verbose=1, epochs=epochs,
          callbacks=get_callbacks(run_name, patience), shuffle=True)
    
model.save('./models/'+run_name)    
print('model saved')
print_time()

Tue Nov 23 01:17:50 2021
############ 



2021-11-23 01:17:50.848441: I tensorflow/core/profiler/lib/profiler_session.cc:126] Profiler session initializing.
2021-11-23 01:17:50.848478: I tensorflow/core/profiler/lib/profiler_session.cc:141] Profiler session started.
2021-11-23 01:17:51.357419: I tensorflow/core/profiler/lib/profiler_session.cc:159] Profiler session tear down.
2021-11-23 01:17:51.361657: I tensorflow/core/profiler/internal/gpu/cupti_tracer.cc:1743] CUPTI activity buffer flushed


Epoch 1/5
    1/11060 [..............................] - ETA: 16:37:36 - loss: 0.2388

2021-11-23 01:18:06.628555: I tensorflow/core/profiler/lib/profiler_session.cc:126] Profiler session initializing.
2021-11-23 01:18:06.628602: I tensorflow/core/profiler/lib/profiler_session.cc:141] Profiler session started.


   39/11060 [..............................] - ETA: 3:45 - loss: 0.1262  

2021-11-23 01:18:07.241172: I tensorflow/core/profiler/lib/profiler_session.cc:66] Profiler session collecting data.
2021-11-23 01:18:07.242941: I tensorflow/core/profiler/internal/gpu/cupti_tracer.cc:1743] CUPTI activity buffer flushed
2021-11-23 01:18:07.271947: I tensorflow/core/profiler/internal/gpu/cupti_collector.cc:673]  GpuTracer has collected 76 callback api events and 74 activity events. 
2021-11-23 01:18:07.274596: I tensorflow/core/profiler/lib/profiler_session.cc:159] Profiler session tear down.
2021-11-23 01:18:07.277315: I tensorflow/core/profiler/rpc/client/save_profile.cc:137] Creating directory: /home/share/TMD_Colab/validation/training/log/final_current/train/plugins/profile/2021_11_23_01_18_07
2021-11-23 01:18:07.279159: I tensorflow/core/profiler/rpc/client/save_profile.cc:143] Dumped gzipped tool data for trace.json.gz to /home/share/TMD_Colab/validation/training/log/final_current/train/plugins/profile/2021_11_23_01_18_07/precision.trace.json.gz
2021-11-23 01:18:0

11060/11060 [==============================] - 49s 4ms/step - loss: 0.0063 - val_loss: 0.0014
Epoch 2/5
 8196/11060 [=====================>........] - ETA: 9s - loss: 0.0011

KeyboardInterrupt: 

# target sigmoid

In [ ]:
%%time

# run setup
lable = 'targetaff'
run_name = 'final_target'
mld_preprocessing_name = './models/target_norm_layer'
file = 'EIC_aff_targetaff.pkl'
path2Data = '../data/training/'
units = 896
units2 = 256
patience = 50
outputlayer = "sigmoid" 
learning_rate = 1e-4
batch_size = 7400
epochs = 5
classes = 1

# set up logs
working_dir = os.getcwd()
log_dir = os.path.join(working_dir, 'log')
print(log_dir)
if os.path.isdir(log_dir) == 0:
    os.mkdir(log_dir)

# reading data
print('\nYou have selected:',file)
train_features = np.load(path2Data+file, allow_pickle=True)

# phrase data
train_labels = train_features.pop(lable)
print(train_features.head())
print(train_labels)
print(type(train_features))
train_features = tf.convert_to_tensor(train_features)
print(type(train_features))
train_labels = tf.convert_to_tensor(train_labels)

# preprcessing layer
input_shape = train_features.shape[1:]
print(input_shape)

model = keras.models.load_model(mld_preprocessing_name)
normalizer = model.layers.pop(0)

/home/share/TMD_Colab/validation/training/log

You have selected: EIC_aff_targetaff.pkl


In [ ]:
%%time

# set up model
def get_callbacks(name, patience):
    return [
    tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=patience),
    tf.keras.callbacks.TensorBoard(log_dir+'/'+name),
    tf.keras.callbacks.ModelCheckpoint(
        filepath=working_dir+'/checkpoints/'+ name + '_{epoch:02d}-{val_loss:.4f}.ckpt',
        save_weights_only=True,
        monitor='val_loss',
        mode='auto',
        save_best_only=True),
    ]

inputs = keras.Input(shape=input_shape)
norm = normalizer(inputs)
hidden_layer1 = layers.Dense(units, activation="relu")(norm)
hidden_layer2 = layers.Dense(units2, activation="relu")(hidden_layer1)
outputs = layers.Dense(classes, activation=outputlayer)(hidden_layer2)
model = keras.Model(inputs, outputs)

# with tf.device(GPU_device):
model.compile(loss='mean_squared_error',
              optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate))

model.summary()

In [ ]:
%%time

print_time()
# training 
model.fit(x=train_features, y=train_labels, validation_split=0.2,
          batch_size=batch_size, verbose=1, epochs=epochs,
          callbacks=get_callbacks(run_name, patience), shuffle=True)
    
model.save('./models/'+run_name)    
print('model saved')
print_time()

# Soft sigmoid

In [ ]:
%%time

# run setup
lable = 'softaff'
run_name = 'final_soft'
mld_preprocessing_name = './models/soft_norm_layer' 
file = 'EIC_aff_softaff.pkl'
path2Data = '../data/training/'
units = 576
units2 = 736
patience = 50
outputlayer = "sigmoid" 
learning_rate = 1e-4
batch_size = 7400
epochs = 500
classes = 1

# set up logs
working_dir = os.getcwd()
log_dir = os.path.join(working_dir, 'log')
print(log_dir)
if os.path.isdir(log_dir) == 0:
    os.mkdir(log_dir)

# reading data
print('\nYou have selected:',file)
train_features = np.load(path2Data+file, allow_pickle=True)

# phrase data
train_labels = train_features.pop(lable)
print(train_features.head())
print(train_labels)
print(type(train_features))
train_features = tf.convert_to_tensor(train_features)
print(type(train_features))
train_labels = tf.convert_to_tensor(train_labels)

# preprcessing layer
input_shape = train_features.shape[1:]
print(input_shape)

model = keras.models.load_model(mld_preprocessing_name)
normalizer = model.layers.pop(0)

In [ ]:
%%time

# set up model
def get_callbacks(name, patience):
    return [
    tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=patience),
    tf.keras.callbacks.TensorBoard(log_dir+'/'+name),
    tf.keras.callbacks.ModelCheckpoint(
        filepath=working_dir+'/checkpoints/'+ name + '_{epoch:02d}-{val_loss:.4f}.ckpt',
        save_weights_only=True,
        monitor='val_loss',
        mode='auto',
        save_best_only=True),
    ]

inputs = keras.Input(shape=input_shape)
norm = normalizer(inputs)
hidden_layer1 = layers.Dense(units, activation="relu")(norm)
hidden_layer2 = layers.Dense(units2, activation="relu")(hidden_layer1)
outputs = layers.Dense(classes, activation=outputlayer)(hidden_layer2)
model = keras.Model(inputs, outputs)


model.compile(loss='mean_squared_error',
              optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate))

model.summary()


In [ ]:
%%time

print_time()
# training 

model.fit(x=train_features, y=train_labels, validation_split=0.2,
          batch_size=batch_size, verbose=1, epochs=epochs,
          callbacks=get_callbacks(run_name, patience), shuffle=True)
    
model.save('./models/'+run_name)    
print('model saved')
print_time()